## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```
---

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```
---

In [1]:
# SOLITION Q1
import numpy as np
from fastembed import TextEmbedding

model = TextEmbedding("jinaai/jina-embeddings-v2-small-en")
query = "I just discovered the course. Can I join now?"
embeddings = list(model.embed([query]))
embedding_array = np.array(embeddings[0])
EMBEDDING_DIMENSIONALITY = 512

# Verify the shape is (512,)
print("Embedding shape:", embedding_array.shape)

# Find the minimal value
min_value = np.min(embedding_array)
print("Minimal value in embedding:", min_value)

# Verify normalization
norm = np.linalg.norm(embedding_array)
print("Vector norm:", norm) 

# Cosine similarity with itself
cos_sim = np.dot(embedding_array, embedding_array)
print("Cosine similarity with itself:", cos_sim)

Embedding shape: (512,)
Minimal value in embedding: -0.11726373885183883
Vector norm: 1.0
Cosine similarity with itself: 1.0000000000000002


## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9
---

In [2]:
doc = 'Can I still join the course after the start date?'

# Generate embeddings (returns generators)
doc_embedding = next(iter(model.embed([doc]))) 
query_embedding = next(iter(model.embed([query])))

# Convert to numpy arrays
doc_vec = np.array(doc_embedding)
query_vec = np.array(query_embedding)

# Calculate cosine similarity (dot product of normalized vectors)
cosine_similarity = np.dot(query_vec, doc_vec)
print(f"Cosine similarity: {cosine_similarity:.4f}")

Cosine similarity: 0.9009


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [3]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

---

In [4]:
# Prepare document texts
doc_texts = [doc['text'] for doc in documents]

# Get all document embeddings at once (more efficient)
doc_embeddings = np.array(list(model.embed(doc_texts)))

# Calculate cosine similarities (matrix multiplication)
similarities = doc_embeddings.dot(query_vec)

# Find index of highest similarity
best_match_idx = np.argmax(similarities)

print(f"Document similarities: {similarities}")
print(f"Highest similarity at index: {best_match_idx}")

Document similarities: [0.76296845 0.81823782 0.80853974 0.71330788 0.73044992]
Highest similarity at index: 1


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [5]:
# Create full_text field and embed
full_texts = [f"{doc['question']} {doc['text']}" for doc in documents]
full_text_embeddings = np.array(list(model.embed(full_texts)))

# Calculate cosine similarities
similarities = full_text_embeddings.dot(query_vec)
best_match_idx = np.argmax(similarities)

print("Similarities with full_text:", np.round(similarities, 3))
print("Highest similarity at index:", best_match_idx)

Similarities with full_text: [0.851 0.844 0.841 0.776 0.809]
Highest similarity at index: 0


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [6]:
model = TextEmbedding("BAAI/bge-small-en")

sample_embedding = next(model.embed(["test"]))
print(f"Embedding size: {len(sample_embedding)}")

Embedding size: 384


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [7]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

In [8]:
text = doc['question'] + ' ' + doc['text']

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67

In [9]:
from qdrant_client import QdrantClient, models

texts = [f"{doc['question']} {doc['text']}" for doc in documents]

In [10]:
client = QdrantClient("http://localhost:6333")
collection_name = "ml_zoomcamp_faq"

In [11]:
# Create collection with 384-dimensional vectors
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimension of BGE-small-en
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_56416/1009804175.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [13]:
# Add documents to Qdrant
batch_size = 50
points = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_docs = documents[i:i+batch_size]

    # Generate embeddings for batch
    batch_embeddings = list(model.embed(batch_texts))
    
    for j, vec in enumerate(batch_embeddings):
        idx = i + j
        doc = batch_docs[j]
        
        points.append(models.PointStruct(
            id=idx,
            vector=vec.tolist(),
            payload={
                'question': doc['question'],
                'text': doc['text'],
                'section': doc['section'],
                'course': doc['course']
            }
        ))

    # Insert batch
    client.upsert(
        collection_name=collection_name,
        points=points
    )
    points = []  

In [14]:
# Query with Q1 question
query_vector = next(model.embed([query])).tolist()

search_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1
)  

if search_results:
    top_score = search_results[0].score
    print(f"Highest score: {top_score:.4f}")
else:
    print("No results found")

Highest score: 0.8703


/tmp/ipykernel_56416/4102702814.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
